In [3]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as tt
import json
import re
import string
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import Vocab
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import wandb

from torch.optim import Adam

import matplotlib.pyplot as plt
import numpy as np
#from vae_utils import get_vector_from_label, add_vector_to_images, morph_faces

print(torch.__version__)
print(torchvision.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
device

2.0.1
0.15.1


device(type='mps')

In [4]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 128
EPOCHS = 25

In [6]:
import zipfile

zip_file = '/Users/parkermoesta/Library/Mobile Documents/com~apple~CloudDocs/Generative Models/LSTM/recipe_generator_LSTM/epirecipes.zip'
extract_dir = '/Users/parkermoesta/Library/Mobile Documents/com~apple~CloudDocs/Generative Models/LSTM/recipe_generator_LSTM/data'

# Open the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    # Extract all the contents of the zip file to the specified directory
    zip_ref.extractall(extract_dir)

print("Extraction complete.")

Extraction complete.


## Load the data

In [7]:
# Load the full dataset
with open("/Users/parkermoesta/Library/Mobile Documents/com~apple~CloudDocs/Generative Models/LSTM/recipe_generator_LSTM/data/full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [8]:
# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

## Tokenization

In [9]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]
text_data[9]

'Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas | Chop enough parsley leaves to measure 1 tablespoon ; reserve . Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan , covered , 5 minutes . Meanwhile , sprinkle gelatin over water in a medium bowl and let soften 1 minute . Strain broth through a fine - mesh sieve into bowl with gelatin and stir to dissolve . Season with salt and pepper . Set bowl in an ice bath and cool to room temperature , stirring . Toss ham with reserved parsley and divide among jars . Pour gelatin on top and chill until set , at least 1 hour . Whisk together mayonnaise , mustard , vinegar , 1 / 4 teaspoon salt , and 1 / 4 teaspoon pepper in a large bowl . Stir in celery , cornichons , and potatoes . Pulse peas with marjoram , oil , 1 / 2 teaspoon pepper , and 1 / 4 teaspoon salt in a food processor to a coarse mash . Layer peas , then potato salad , over ham . '

In [10]:
class TextDataset(Dataset):
    def __init__(self, text_data, tokenizer):
        self.text_data = text_data
        self.tokenizer = tokenizer
    
    def __getitem__(self, idx):
        tokens = self.tokenizer(self.text_data[idx])
        tokens = tokens[:200]  # Clip to maximum length of 200
        return tokens
    
    def __len__(self):
        return len(self.text_data)

In [11]:
tokenizer = get_tokenizer('basic_english')

In [12]:
def yield_tokens(data_iter):
    for text_str in data_iter:
        yield tokenizer(text_str)

In [13]:
vocab = build_vocab_from_iterator(yield_tokens(text_data), max_tokens=VOCAB_SIZE, specials=['<pad>', '<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [14]:
def collate_batch(batch):
    stoi = vocab.get_stoi()
    batch = [torch.tensor([stoi.get(token, stoi['<unk>']) for token in tokens]) for tokens in batch]
    batch = pad_sequence(batch, batch_first=True, padding_value=stoi['<pad>'])
    x = batch[:, :-1]
    y = batch[:, 1:]
    return x, y

In [15]:
text_ds = TextDataset(text_data, tokenizer)
text_loader = DataLoader(text_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [15]:
# Get the first batch from the DataLoader
for x, y in text_loader:
    break

# Convert indices back to tokens
def indices_to_tokens(indices):
    itos = vocab.get_itos()
    return ' '.join([itos[index] for index in indices])

# Print the first few sentences in the batch
for i in range(5):  # Change this number to print more or fewer sentences
    print("Input (x):", indices_to_tokens(x[i]))
    print("Target (y):", indices_to_tokens(y[i]))
    print("---")

Input (x): recipe for doubles | in small bowl , stir together water , sugar , and yeast . let stand until foamy , about 5 or 6 minutes . in large bowl whisk together flour , salt , turmeric , cumin , and pepper . stir in yeast mixture , then add additional warm water , if needed , until mixture comes together into slightly firm dough . knead dough in bowl 2 minutes , then form into ball and cover with damp cloth . let dough rise in warm , draft - free place until doubled , about 1 hour . if using dried chickpeas , drain and add 6 cups fresh water . simmer until tender , about 1 hour . drain . if using canned chickpeas , drain and rinse well with cold water . in heavy skillet over moderately high heat , heat oil . add onion and sauté until translucent . add garlic and sauté 1 minute more . mix in curry powder and sauté 30 seconds , then add 1 / 4 cup water . stir in chickpeas , cover , and simmer 5 minutes . add 1 cup water and cumin . season
Target (y): for doubles | in small bowl , st

## Building the LSTM model

In [16]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=2)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x)
        return self.log_softmax(x)

In [17]:
model = LSTMModel(VOCAB_SIZE, EMBEDDING_DIM, N_UNITS).to(device)

print(model)

LSTMModel(
  (embedding): Embedding(10000, 100)
  (lstm): LSTM(100, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=10000, bias=True)
  (log_softmax): LogSoftmax(dim=2)
)


VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

In [19]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="LTSM - Recipe Generation - fixed gen_v2",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.002,
    "architecture": "RNN-LSTM",
    "dataset": "Recipes",
    "epochs": 50,
    "batch_size": 128,
    "latent_dim": 100,
    "BatchNorm": True,
    "Dropout": True,
    "Normalize": True,
    "Gradient Penalty": False,
    "loss function": "NLLLoss",
    "activation": "LeakyReLU",
    "notes": "Inc learning on gen, dec on disc. incerasing gradient penalty to 12",
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: parkermoesta. Use `wandb login --relogin` to force relogin


In [18]:


# Define the loss function and optimizer
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())

# Define the number of epochs
EPOCHS = 50  

In [19]:
from tqdm import tqdm

class TextGenerator:
    def __init__(self, vocab, top_k=10):
        self.vocab = vocab
        self.top_k = top_k

    def sample_from(self, logits, temperature):
        probs = F.softmax(logits / temperature, dim=-1).cpu().numpy()
        return np.random.choice(len(probs), p=probs)

    def generate(self, model, device, start_prompt, max_tokens, temperature):
        model.eval()

        tokens = [self.vocab.get_stoi()[token] for token in start_prompt.split()]
        tokens = torch.LongTensor(tokens).unsqueeze(0).to(device)

        with torch.no_grad():
            for _ in range(max_tokens):
                output = model(tokens)
                next_token_logits = output[0, -1, :]
                next_token = self.sample_from(next_token_logits, temperature)
                tokens = torch.cat([tokens, torch.LongTensor([[next_token]]).to(device)], dim=1)

        generated_text = ' '.join(self.vocab.get_itos()[token] for token in tokens[0])
        return generated_text


def train(model, optimizer, text_loader, device, start_epoch, num_epochs, checkpoint_path=None, log_wandb=True):
    # Load the checkpoint if it exists
    if checkpoint_path is not None and log_wandb:
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print('Loaded model from checkpoint')
        
    # Define the loss function
    loss_fn = nn.NLLLoss()

    # Training loop
    for epoch in tqdm(range(start_epoch, num_epochs)):
        model.train()  # Ensure the model is in training mode
        for i, (x, y) in enumerate(text_loader):
            # Move the data to the device
            x = x.to(device)
            y = y.to(device)

            # Forward pass
            y_pred = model(x)

            # Compute the loss
            loss = loss_fn(y_pred.reshape(-1, VOCAB_SIZE), y.reshape(-1))

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            
            # Compute gradient norm and clip gradients
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            
            optimizer.step()

            # Print loss every 100 batches
            if i % 100 == 0:
                print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')

            # Log loss and perplexity to Weights & Biases
            if log_wandb:
                wandb.log({
                    "Epoch": epoch,
                    "Batch": i,
                    "Loss": loss.item(),
                    "Perplexity": torch.exp(loss).item(),
                    "Gradient Norm": grad_norm.item()
                })

        text_generator = TextGenerator(vocab)
        generated_text = text_generator.generate(model, device, "recipe for", max_tokens=100, temperature=1.0)
        print(generated_text)
        if log_wandb:
            wandb.log({"Generated Text": wandb.Html(f"<pre>{generated_text}</pre>", inject=False)})

        # Save a checkpoint after each epoch
        if log_wandb:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch
            }, f'checkpoint_epoch_{epoch}.pth')

            # log the checkpoint to wandb
            wandb.save(f'checkpoint_epoch_{epoch}.pth')



In [25]:
start_epoch = 50
checkpoint_path = f'checkpoint_epoch_{start_epoch - 1}.pth'

train(model, optimizer, text_loader, device, start_epoch=50, num_epochs=100, checkpoint_path=checkpoint_path, log_wandb=True)

Loaded model from checkpoint


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 50, Batch 0, Loss: 1.3963435888290405
Epoch 50, Batch 100, Loss: 1.3973511457443237


  2%|▏         | 1/50 [00:41<33:29, 41.01s/it]

recipe for roast quail with haricots verts and eggplant with savoy tomatoes | 1 . prepare a gas grill with grill for grilling . grill the bell peppers for 1 - 3 or 3 or a flame , covered with a weighted and ensure hot yellow to coat the fish well with the marinade , for 2 - oz . 2 inch brown oil marinade inside a large cast iron skillet , then reduce temperature to 200°f . oil the grill pan or heat off 350°f . the chimichurri tofu aside . starting from the pans and boil for a minute or
Epoch 51, Batch 0, Loss: 1.4483486413955688
Epoch 51, Batch 100, Loss: 1.507794976234436


...

recipe for wakame beignets tostada with lemon cream | in a large pitcher place mango mixture in 2 - quart terrine or coffee bottle , making sure that enjoy . strain . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Epoch 52, Batch 0, Loss: 1.4285203218460083
Epoch 52, Batch 100, Loss: 1.4886155128479004


  6%|▌         | 3/50 [02:00<31:20, 40.01s/it]

recipe for zucchini seed and corn muffins | preheat oven to 375°f . cut away dark flesh of white core but about 1 slice into 1 - inch pieces . place 1 tablespoon filling in bottom of shell . carefully remove paper from oranges , leaving deep skin side in same manner . in butter 3 - to 4 top with 4 - inch slices , making striped line each baking sheets with parchment paper . mound flour in top of double layer of large rimmed baking sheet . using electric mixer , beat 1 1 / 2 cups cornmeal and cinnamon
Epoch 53, Batch 0, Loss: 1.333232045173645
Epoch 53, Batch 100, Loss: 1.5901602506637573


  8%|▊         | 4/50 [02:39<30:18, 39.54s/it]

recipe for tortellini in chicken stock | stir together water , vinegar , garlic , ginger , and 2 cups water . cover and refrigerate slightly several hours for 15 minutes . place ground cloves on a work surface of plastic wrap . 3 . for a charcoal grill , cook an coals on a gas grill , turning often on bottom of grill and grate or place a bit before using a heavy cleaver or small amount of dough from water ( don ' t worry if some part of briquettes ) . grill corn , and thread on a top
Epoch 54, Batch 0, Loss: 1.3600329160690308
Epoch 54, Batch 100, Loss: 1.4815459251403809


In [20]:
# loading model from checkpoint
checkpoint_path = '/Users/parkermoesta/Library/Mobile Documents/com~apple~CloudDocs/Generative Models/LSTM/recipe_generator_LSTM/checkpoint_epoch_99.pth'
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
print('Loaded model from checkpoint')

Loaded model from checkpoint


In [32]:
text_generator = TextGenerator(vocab=vocab, top_k=10)
generated_text = text_generator.generate(model=model, device=device, start_prompt="recipe for roasted vegetables | chop 1 /", max_tokens=100, temperature=0.5)

print(f"\nGenerated Text: {generated_text}")



Generated Text: recipe for roasted vegetables | chop 1 / 2 cup onion and reserve for another use . place garlic in a large , heavy - bottomed saucepan over medium - high heat . add onions and sauté until tender , about 5 minutes . add garlic and stir - fry until water evaporates , about 2 minutes . add wine and bring to boil . reduce heat to medium and simmer , covered , until vegetables are tender , about 20 minutes . uncover and cook until liquid is reduced by half , about 30 minutes . add wine and boil until reduced by half , about 1
